## Week8 Basic Homework
- corpus 사용하여 facebook/opt-350m 학습

In [1]:
import os
import sys
import json
import torch
import wandb
import logging
import evaluate
import transformers
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import datasets
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)

from peft import (
    get_peft_config,
    get_peft_model,
    LoraConfig,
    TaskType
)

from trl import (
    SFTConfig,
    SFTTrainer,
    DataCollatorForCompletionOnlyLM
)

from transformers.trainer_utils import get_last_checkpoint

In [2]:
#device setting
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.backends.cuda.is_built():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

mps


## [MY CODE] Model Load (opt-350m)

In [3]:
# facebook - 2k tokens
model_name = "facebook/opt-350m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype=torch.bfloat16
)

In [4]:
lora_r: int = 8
lora_dropout: float = 0.1
lora_alpha: int = 32

target_modules = set()

for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        names = name.split('.')
        target_modules.add(names[0] if len(names) == 1 else names[-1])

if "lm_head" in target_modules:  # needed for 16-bit
    target_modules.remove("lm_head")

target_modules = list(target_modules)

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=target_modules
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 3,563,520 || all params: 334,759,936 || trainable%: 1.0645


## [MY CODE] Data Load (corpus)

In [6]:
with open("corpus_small.json", "r", encoding="utf-8") as f:
    corpus = json.load(f)

df = pd.DataFrame(corpus)

train_df, test_df = train_test_split(df, test_size = 0.2, random_state = 42)

ds_train = datasets.Dataset.from_pandas(train_df, preserve_index=False)
ds_test = datasets.Dataset.from_pandas(test_df, preserve_index=False)

data_files = datasets.DatasetDict({"train":ds_train,
                                   "test":ds_test})

data_files

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 110
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 28
    })
})

## [MY CODE] 학습 준비 - training arguments, data preprocess function, collator

In [7]:
# # wandb 접속
wandb.init(project='Hanghae99')
wandb.run.name = 'fb-lora8-finetuning'

logger = logging.getLogger()

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

wandb: Currently logged in as: imsta (imsta-hub) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
training_args = SFTConfig(output_dir=os.path.join(os.getcwd(),'trained_model/fb_8'),
                        learning_rate=1e-4,
                        per_device_train_batch_size=2,
                        per_device_eval_batch_size=2,
                        num_train_epochs=10,
                        weight_decay=0.01,
                        eval_strategy="epoch",
                        save_strategy="epoch",
                        load_best_model_at_end=True,
                        report_to = "wandb",
                        push_to_hub=False,
                        )

# logger 설정
if training_args.should_log:
    transformers.utils.logging.set_verbosity_info()  # log level을 INFO로 변경

# log level: 10 DEBUG, 20 INFO, 30 WARNING, 40 ERROR, 50 CRITICAL
log_level = training_args.get_process_log_level()

# 우리가 가지고 있는 logger와 HuggingFace의 logger의 log level 설정
logger.setLevel(log_level)
transformers.utils.logging.set_verbosity(log_level)

# 기타 HuggingFace logger option들을 설정
transformers.utils.logging.enable_default_handler() # logger 기능 활성화
transformers.utils.logging.enable_explicit_format() # 포맷 설정: [LEVELNAME|FILENAME|LINE NUMBER] TIME >> MESSAGE

In [9]:
def formatting_prompts_func(src_df):
    output_texts = []
    for i in range(len(src_df['input'])):
        text = f"{src_df['input'][i]}\n{src_df['output'][i]}"
        output_texts.append(text)
    return output_texts

In [10]:
tokenizer.padding_side = "right"

response_template = "Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [11]:
trainer = SFTTrainer(
    model,
    train_dataset=ds_train,
    args=training_args,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    eval_dataset=ds_test
)
#trainer.train()

checkpoint = None
last_checkpoint = get_last_checkpoint(training_args.output_dir)  # 만약 output_dir에 checkpoint가 남아있으면 이를 사용하고, 없으면 None이 return됩니다.
if training_args.resume_from_checkpoint is not None:  # output_dir이 아닌 다른 위치에서의 checkpoint를 resume_from_checkpoint로 지정할 수 있습니다.
    checkpoint = training_args.resume_from_checkpoint
else:  # 아니면 last_checkpoint로 checkpoint를 지정합니다.
    checkpoint = last_checkpoint

train_result = trainer.train()

trainer.save_model()

metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

[INFO|configuration_utils.py:695] 2025-02-12 21:47:50,481 >> loading configuration file config.json from cache at /Users/jh/.cache/huggingface/hub/models--facebook--opt-350m/snapshots/08ab08cc4b72ff5593870b5d527cf4230323703c/config.json
[INFO|configuration_utils.py:762] 2025-02-12 21:47:50,484 >> Model config OPTConfig {
  "_name_or_path": "facebook/opt-350m",
  "_remove_final_layer_norm": false,
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "OPTForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "do_layer_norm_before": false,
  "dropout": 0.1,
  "enable_bias": true,
  "eos_token_id": 2,
  "ffn_dim": 4096,
  "hidden_size": 1024,
  "init_std": 0.02,
  "layer_norm_elementwise_affine": true,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "opt",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "prefix": "</s>",
  "torch_dtype": "float16",
  "transformers_version": "4.47.1",
  "us

Map:   0%|          | 0/110 [00:00<?, ? examples/s]

Map:   0%|          | 0/28 [00:00<?, ? examples/s]

[INFO|trainer.py:2362] 2025-02-12 21:47:50,870 >> ***** Running training *****
[INFO|trainer.py:2363] 2025-02-12 21:47:50,870 >>   Num examples = 110
[INFO|trainer.py:2364] 2025-02-12 21:47:50,871 >>   Num Epochs = 10
[INFO|trainer.py:2365] 2025-02-12 21:47:50,871 >>   Instantaneous batch size per device = 4
[INFO|trainer.py:2368] 2025-02-12 21:47:50,871 >>   Total train batch size (w. parallel, distributed & accumulation) = 4
[INFO|trainer.py:2369] 2025-02-12 21:47:50,872 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:2370] 2025-02-12 21:47:50,872 >>   Total optimization steps = 280
[INFO|trainer.py:2371] 2025-02-12 21:47:50,874 >>   Number of trainable parameters = 3,563,520
[INFO|integration_utils.py:811] 2025-02-12 21:47:50,877 >> Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a differen

  0%|          | 0/280 [00:00<?, ?it/s]

...

{'eval_loss': 1.9375, 'eval_runtime': 13.7059, 'eval_samples_per_second': 2.043, 'eval_steps_per_second': 0.511, 'epoch': 10.0}


[INFO|configuration_utils.py:695] 2025-02-12 22:43:34,662 >> loading configuration file config.json from cache at /Users/jh/.cache/huggingface/hub/models--facebook--opt-350m/snapshots/08ab08cc4b72ff5593870b5d527cf4230323703c/config.json
[INFO|configuration_utils.py:762] 2025-02-12 22:43:34,665 >> Model config OPTConfig {
  "_name_or_path": "opt-350m",
  "_remove_final_layer_norm": false,
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "OPTForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "do_layer_norm_before": false,
  "dropout": 0.1,
  "enable_bias": true,
  "eos_token_id": 2,
  "ffn_dim": 4096,
  "hidden_size": 1024,
  "init_std": 0.02,
  "layer_norm_elementwise_affine": true,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "opt",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "prefix": "</s>",
  "torch_dtype": "float16",
  "transformers_version": "4.47.1",
  "use_cache":

{'train_runtime': 3344.2065, 'train_samples_per_second': 0.329, 'train_steps_per_second': 0.084, 'train_loss': 1.8474888392857143, 'epoch': 10.0}


[INFO|configuration_utils.py:695] 2025-02-12 22:43:35,692 >> loading configuration file config.json from cache at /Users/jh/.cache/huggingface/hub/models--facebook--opt-350m/snapshots/08ab08cc4b72ff5593870b5d527cf4230323703c/config.json
[INFO|configuration_utils.py:762] 2025-02-12 22:43:35,695 >> Model config OPTConfig {
  "_name_or_path": "opt-350m",
  "_remove_final_layer_norm": false,
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "OPTForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "do_layer_norm_before": false,
  "dropout": 0.1,
  "enable_bias": true,
  "eos_token_id": 2,
  "ffn_dim": 4096,
  "hidden_size": 1024,
  "init_std": 0.02,
  "layer_norm_elementwise_affine": true,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "opt",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "prefix": "</s>",
  "torch_dtype": "float16",
  "transformers_version": "4.47.1",
  "use_cache":

***** train metrics *****
  epoch                    =       10.0
  total_flos               =  1788434GF
  train_loss               =     1.8475
  train_runtime            = 0:55:44.20
  train_samples_per_second =      0.329
  train_steps_per_second   =      0.084
